In [1]:
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
import sys
import utils
import base64
import pandas as pd

In [2]:
client = OpenAI()

In [3]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Model 1

In [35]:
road_grading_standards = """Road Condition Rating System (Pavement Surface Condition Index - PSCI):
        10: New roads, no defects.
        9: Very few defects (<10% surface issues).
        8: Minor defects (10-30% surface issues), no structural problems.
        7: Noticeable surface defects (>30%), no structural problems.
        6: Some cracking (<20%), good patching, minor distortions.
        5: Significant cracking (>20%), fair patching, notable distortions.
        4: Moderate structural issues (5-25%), frequent potholes.
        3: Major structural issues (25-50%), frequent potholes.
        2: Severe structural issues (>50%), many potholes.
        1: Complete road failure, extensive damage.
        """

In [36]:
def get_road_condition_grade1(folder_path, model):
    # Iterate through each image in the folder
    grade = []
    img_names = []
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            base64_image = encode_image(image_path)
            # Construct the message with the specified instructions
            messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                }
                            },
                            {"type": "text", "text": "Focusing on the asphalt road, do you see any cracks, alligator cracking, potholes, ravelling, bleeding, rutting, surface distortion, edge breakup, patching, etc?"},
                            {"type": "text", "text":  road_grading_standards},
                            {"type": "text", "text": "Please provide a grade for the road condition based on the provided standards, only a single grade number. Please note that when identifying road anomalies, exclude the effects of shadows and water stains caused by light.Response me with only one word"},
                        ]
                    },
                ]
            
            response = client.chat.completions.create(
                model= model,  # Note: This is a conceptual model name
                messages=messages,
                max_tokens=300,
                temperature=0
            )
            
            try:
                grade_value = int(response.choices[0].message.content)
                grade.append(grade_value)
                img_names.append(image_name)
            except ValueError:
                grade_value = 11
                grade.append(grade_value)
                img_names.append(image_name)
                print(f"Invalid response for {image_name}: {response.choices[0].message.content}")
    return (img_names, grade)

In [37]:
folder_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\imgs_all"
output = get_road_condition_grade1(folder_path,"gpt-4o")

df = pd.DataFrame({'Name': output[0], 'Model': output[1]})
df['Name'] = df['Name'].str.extract('(\d+)')
csv_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\RC_Model1_o_09211606.csv"#RC_Model2_o6_08252333.csv
df.to_csv(csv_path, index=False)

# Model 2

In [38]:
road_grading_standards = """Road Condition Rating System (Pavement Surface Condition Index - PSCI):
        10: New roads, no defects.
        9: Very few defects (<10% surface issues).
        8: Minor defects (10-30% surface issues), no structural problems.
        7: Noticeable surface defects (>30%), no structural problems.
        6: Some cracking (<20%), good patching, minor distortions.
        5: Significant cracking (>20%), fair patching, notable distortions.
        4: Moderate structural issues (5-25%), frequent potholes.
        3: Major structural issues (25-50%), frequent potholes.
        2: Severe structural issues (>50%), many potholes.
        1: Complete road failure, extensive damage.
        """

In [39]:
def get_road_condition_grade1(folder_path, model):
    # Iterate through each image in the folder
    grade = []
    img_names = []
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            base64_image = encode_image(image_path)
            # Construct the message with the specified instructions
            messages=[
                      {
                        "role": "system",
                        "content":"You are a pavement engineer.Please provide a grade for the road condition based on the\
                             provided standards and image. Response me with only one number(grade)"},
                      {
                        "role": "user",
                        "content": [
                            { "type": "image_url","image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                            {"type": "text", "text":  "Please pay attention to any anomalies on the asphalt pavement"},
                            {"type": "text", "text":  road_grading_standards},
                            {"type": "text", "text": "Using the provided road condition grading criteria determine the \
                            grade of the road surface in the image"},

                        ]
                    },
                ]
            
            response = client.chat.completions.create(
                model= model,  # Note: This is a conceptual model name
                messages=messages,
                max_tokens=300,
                temperature=0
            )
            
            try:
                grade_value = int(response.choices[0].message.content)
                grade.append(grade_value)
                img_names.append(image_name)
            except ValueError:
                grade_value = 11
                grade.append(grade_value)
                img_names.append(image_name)
                print(f"Invalid response for {image_name}: {response.choices[0].message.content}")
    return (img_names, grade)

In [40]:
folder_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\imgs_all"
output = get_road_condition_grade1(folder_path,"gpt-4o")

df = pd.DataFrame({'Name': output[0], 'Model': output[1]})
df['Name'] = df['Name'].str.extract('(\d+)')
csv_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\RC_Model2_o_09211606.csv"#RC_Model2_o6_08252333.csv
df.to_csv(csv_path, index=False)

# Model 3

In [41]:
road_grading_standards = """Road Condition Rating System (Pavement Surface Condition Index - PSCI):
    Rating 10 (Excellent Road Condition - Routine Maintenance): Newly constructed or recently overlaid roads with no visible defects.
    Rating 9 (Very Good Road Condition - Routine Maintenance): Very few visible defects; less than 10% surface defects (ravelling and bleeding).
    Rating 8 (Good Road Condition - Restoration of Skid Resistance): 10% to 30% surface with surface defects; no structural distresses present.
    Rating 7 (Good Road Condition - Restoration of Skid Resistance): More than 30% of surface with surface defects (ravelling or bleeding); no structural distresses.
    Rating 6 (Fair Road Condition - Surface Restoration): Less than 20% other cracking, patching in good condition, some surface distortion present.
    Rating 5 (Fair Road Condition - Surface Restoration): More than 20% surface with other cracking, patching in fair condition, significant surface distortions.
    Rating 4 (Poor Road Condition - Structural Overlay): 5% to 25% of surface with structural distresses (rutting, alligator cracking, poor patching), frequent potholes.
    Rating 3 (Poor Road Condition - Structural Overlay): 25% to 50% of surface with significant structural distresses, more frequent potholes.
    Rating 2 (Very Poor  Road Condition - Road Reconstruction): More than 50% of surface with severe structural distresses, severe rutting, many potholes.
    Rating 1 (Failed Poor  Road Condition - Road Reconstruction): Complete road failure, many large and deep potholes, extensive loss of pavement surface.
    
    Defect Categories and Indicators:
    Surface Defects: Include ravelling (loss of binder and aggregate from the surface) and bleeding (excess bituminous material on the surface).
    Pavement Deformation: Encompasses rutting (permanent deformation in wheel paths) and surface distortion (depressions, bumps).
    Cracks: Categorized into alligator cracking (series of interconnected cracks), edge breakup/cracking, and other cracking (longitudinal, transverse, reflection, slippage).
    Surface Openings: Include patching (repaired areas indicating previous pavement defects) and potholes (bowl-shaped depressions exposing underlying layers).
"""

In [42]:
def get_road_condition_grade1(folder_path, model):
    # Iterate through each image in the folder
    grade = []
    img_names = []
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            base64_image = encode_image(image_path)
            # Construct the message with the specified instructions
            messages=[
                      {
                        "role": "system",
                        "content":"You are a pavement engineer.Please provide a grade for the road condition based on the\
                             provided standards and image. Response me with only one number(grade)"},
                      {
                        "role": "user",
                        "content": [
                            { "type": "image_url","image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                            {"type": "text", "text":  "Please pay attention to any anomalies on the asphalt pavement"},
                            {"type": "text", "text":  road_grading_standards},
                            {"type": "text", "text": "Using the provided road condition grading criteria determine the \
                            grade of the road surface in the image"},

                        ]
                    },
                ]
            
            response = client.chat.completions.create(
                model= model,  # Note: This is a conceptual model name
                messages=messages,
                max_tokens=300,
                temperature=0
            )
            
            try:
                grade_value = int(response.choices[0].message.content)
                grade.append(grade_value)
                img_names.append(image_name)
            except ValueError:
                grade_value = 11
                grade.append(grade_value)
                img_names.append(image_name)
                print(f"Invalid response for {image_name}: {response.choices[0].message.content}")
    return (img_names, grade)

In [43]:
folder_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\imgs_all"
output = get_road_condition_grade1(folder_path,"gpt-4o")

df = pd.DataFrame({'Name': output[0], 'Model': output[1]})
df['Name'] = df['Name'].str.extract('(\d+)')
csv_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\RC_Model3_o_09211606.csv"#RC_Model2_o6_08252333.csv
df.to_csv(csv_path, index=False)

# Model 4

In [8]:
road_grading_standards = """Road Condition Rating System (Pavement Surface Condition Index - PSCI):
    Rating 10 (Excellent Road Condition - Routine Maintenance): Newly constructed or recently overlaid roads with no visible defects.
    Rating 9 (Very Good Road Condition - Routine Maintenance): Very few visible defects; less than 10% surface defects (ravelling and bleeding).
    Rating 8 (Good Road Condition - Restoration of Skid Resistance): 10% to 30% surface with surface defects; no structural distresses present.
    Rating 7 (Good Road Condition - Restoration of Skid Resistance): More than 30% of surface with surface defects (ravelling or bleeding); no structural distresses.
    Rating 6 (Fair Road Condition - Surface Restoration): Less than 20% other cracking, patching in good condition, some surface distortion present.
    Rating 5 (Fair Road Condition - Surface Restoration): More than 20% surface with other cracking, patching in fair condition, significant surface distortions.
    Rating 4 (Poor Road Condition - Structural Overlay): 5% to 25% of surface with structural distresses (rutting, alligator cracking, poor patching), frequent potholes.
    Rating 3 (Poor Road Condition - Structural Overlay): 25% to 50% of surface with significant structural distresses, more frequent potholes.
    Rating 2 (Very Poor  Road Condition - Road Reconstruction): More than 50% of surface with severe structural distresses, severe rutting, many potholes.
    Rating 1 (Failed Poor  Road Condition - Road Reconstruction): Complete road failure, many large and deep potholes, extensive loss of pavement surface.
    
    Defect Categories and Indicators:
    Surface Defects: Include ravelling (loss of binder and aggregate from the surface) and bleeding (excess bituminous material on the surface).
    Pavement Deformation: Encompasses rutting (permanent deformation in wheel paths) and surface distortion (depressions, bumps).
    Cracks: Categorized into alligator cracking (series of interconnected cracks), edge breakup/cracking, and other cracking (longitudinal, transverse, reflection, slippage).
    Surface Openings: Include patching (repaired areas indicating previous pavement defects) and potholes (bowl-shaped depressions exposing underlying layers).
"""

In [9]:
def get_road_condition_grade1(folder_path, model):
    # Iterate through each image in the folder
    grade = []
    img_names = []
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            base64_image = encode_image(image_path)
            # Construct the message with the specified instructions
            messages=[
                    {
                        "role": "system",
                        "content":"You are a pavement engineer tasked with evaluating road conditions from images.\
                        You will also be provided with road condition grading criteria (road_grading_standards) and image.\
                        Use the following step-by-step instructions to evaluate the road grade.\
                        Step 1 – The user will provide you with road condition grading criteria.\
                        Step 2 - The user will provide you with an image in base64 format, focus solely on the asphalt pavement.\
                        Step 3 - Please pay attention to any anomalies on the asphalt pavement, such as cracks, alligator\
                        cracks, potholes, raveling, bleeding, rutting, surface distortion, edge breakup, patching, etc. \
                        When identifying road anomalies, exclude the effects of shadows and water stains caused by lighting\
                        conditions.\
                        Step 4 - Using the provided road condition grading criteria determine the grade of the road surface in the image.\
                        Step 5 - Respond with a single grade number based on the provided standards. \
                        Respond with only one number."},
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": "I will provide you with road condition grading criteria."},
                            {"type": "text", "text": road_grading_standards},
                            {"type": "text", "text": "I will provide you with an image in base64 format, please focus solely on the asphalt pavement"},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                }
                            },
                            {"type": "text", "text": "Please pay attention to any anomalies on the asphalt pavement, such as cracks, alligator\
                            cracks, potholes, raveling, bleeding, rutting, surface distortion, edge breakup, patching, etc.\
                            When identifying road anomalies, exclude the effects of shadows and water stains caused by lighting\
                            conditions."},
                            {"type": "text", "text": "Using the provided road condition grading criteria determine the \
                            grade of the road surface in the image"},
                            {"type": "text", "text": "Respond with a single grade number based on the provided standards.\
                            Respond with only one number."},
                        ]
                    },
                ]
            
            response = client.chat.completions.create(
                model= model,  # Note: This is a conceptual model name
                messages=messages,
                max_tokens=300,
                temperature=0
            )
            
            try:
                grade_value = int(response.choices[0].message.content)
                grade.append(grade_value)
                img_names.append(image_name)
            except ValueError:
                grade_value = 11
                grade.append(grade_value)
                img_names.append(image_name)
                print(f"Invalid response for {image_name}: {response.choices[0].message.content}")
    return (img_names, grade)

In [10]:
folder_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\imgs_all"
output = get_road_condition_grade1(folder_path,"gpt-4o")

df = pd.DataFrame({'Name': output[0], 'Model': output[1]})
df['Name'] = df['Name'].str.extract('(\d+)')
csv_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\RC_Model4_o_10131120.csv"#RC_Model2_o6_08252333.csv
df.to_csv(csv_path, index=False)

Invalid response for 106.png: I can't evaluate the road condition from the image provided. Please provide the image in base64 format.
Invalid response for 56.png: I can't evaluate the road condition from the image provided. Please provide the image in base64 format.


In [ ]:
09092255

Invalid response for 50.png: Based on the provided image, the asphalt pavement shows some surface defects, including patching and minor cracking. There are no significant structural distresses such as rutting, alligator cracking, or frequent potholes visible.

Given the criteria:

- Less than 20% other cracking, patching in good condition, some surface distortion present.

The road condition appears to fit the description for a rating of 6.

Therefore, the grade of the road surface in the image is:

6

In [ ]:
09092333
Invalid response for 50.png: Based on the provided image, the asphalt pavement shows some surface defects, including patching and minor cracking. There are no significant structural distresses such as rutting, alligator cracking, or frequent potholes visible.

Given the criteria:

- Less than 20% other cracking, patching in good condition, some surface distortion present.

The road condition appears to fit the description for a rating of 6.

So, the grade of the road surface in the image is:

6

In [ ]:
09211344
Invalid response for 50.png: Based on the provided image, the asphalt pavement shows some surface defects, including patching and minor cracking. There are no significant structural distresses such as rutting, alligator cracking, or frequent potholes visible.

Given the criteria:

- Less than 20% other cracking, patching in good condition, some surface distortion present.

The road condition grade for the asphalt pavement in the image is:

6
Invalid response for 64.png: Based on the provided image, the asphalt pavement appears to be in good condition with no visible significant defects such as cracks, potholes, raveling, bleeding, rutting, surface distortion, edge breakup, or patching. The surface looks smooth and well-maintained.

Given the criteria, the road condition seems to fit the description of:

- Rating 10 (Excellent Road Condition - Routine Maintenance): Newly constructed or recently overlaid roads with no visible defects.

Therefore, the grade of the road surface in the image is:

10

In [ ]:
09211517
Invalid response for 50.png: Based on the provided image, the asphalt pavement shows some surface defects, including patching and minor cracking. There are no significant structural distresses such as rutting, alligator cracking, or frequent potholes visible.

Given the criteria:

- Less than 20% other cracking, patching in good condition, some surface distortion present.

The road condition appears to fit the description for a rating of 6.

Therefore, the grade of the road surface in the image is:

6
Invalid response for 64.png: Based on the provided image, the asphalt pavement appears to be in good condition with no visible significant defects such as cracks, potholes, raveling, bleeding, rutting, surface distortion, edge breakup, or patching. The surface looks smooth and well-maintained.

Given the criteria, the road condition seems to fall under:

- Rating 10 (Excellent Road Condition - Routine Maintenance): Newly constructed or recently overlaid roads with no visible defects.

Therefore, the grade of the road surface in the image is:

10

# Model5

In [4]:
road_grading_standards = """
    
    Road Condition Rating System (Pavement Surface Condition Index - PSCI):
    Rating 10
        Description: No visible defects. The road is newly constructed or recently overlaid and in perfect condition.
        Key Characteristics: No cracks, potholes, ravelling, or surface distortions. Surface is smooth and even.
        Expected Defects: None.
    Rating 9
        Description: Minor surface defects such as very small areas of ravelling or bleeding (less than 10% of the surface).
        Key Characteristics: The road may show minor loss of surface texture but is generally smooth and without significant structural damage.
        No structural distresses like cracking, rutting, or edge breakup.
        Expected Defects: Minor ravelling or slight bleeding, but very localized.
    Rating 8
        Description: Moderate surface defects such as ravelling or bleeding affecting 10-30% of the surface. No significant structural damage.
        Key Characteristics: Some surface wear is visible, but the overall road surface remains intact. No major cracks, rutting, or other structural defects.
        Expected Defects: Moderate ravelling or bleeding.
    Rating 7
        Description: Extensive surface defects where more than 30% of the surface shows ravelling or bleeding. The road is old but with little or no structural distress.
        Key Characteristics: Large areas of surface wear or bleeding.
        The road may look aged, but there are no major cracks or structural damage.
        Expected Defects: Extensive ravelling or bleeding over the surface.
    Rating 6
        Description: Moderate presence of other pavement defects such as cracking (less than 20% of the surface), surface distortion, or patching in good condition. No structural distresses like rutting or edge cracking.
        Key Characteristics: Visible surface distortions requiring reduced speeds. Cracking may be present but is not severe.
        Expected Defects: Minor cracks (longitudinal or transverse), some surface distortion, good quality patches.
    Rating 5
        Description: Significant other pavement defects affecting more than 20% of the surface. Some structural distresses like rutting, minor alligator cracking, or edge breakup may be visible, but they are localized.
        Key Characteristics: Frequent cracks, surface distortions, and patches. Minor localized structural damage.
        Expected Defects: Frequent cracking, some local rutting, small areas of alligator cracking or edge cracking.
    Rating 4
        Description: Structural distresses present over 5-25% of the surface, including rutting, alligator cracking, poor patching, or edge breakup. Frequent potholes are likely.
        Key Characteristics: The road requires structural overlay to strengthen it. Visible signs of rutting and cracking.
        Expected Defects: Alligator cracking, edge breakup, rutting, frequent potholes.
    Rating 3
        Description: Extensive structural distress covering 25-50% of the surface, with large areas of rutting, alligator cracking, or edge breakup. Potholes and failed patches are frequent.
        Key Characteristics: The road is significantly damaged and difficult to drive on. Large areas show signs of major defects.
        Expected Defects: Widespread alligator cracking, deep rutting, frequent and large potholes, edge disintegration.
    Rating 2
        Description: Over 50% of the road shows severe structural distresses, including rutting deeper than 75mm, large potholes, and very poor patching. The road is difficult to drive on and in need of full-depth reconstruction.
        Key Characteristics: The road is severely deteriorated. Major portions of the surface are cracked, distorted, or missing.
        Expected Defects: Severe rutting, extensive alligator cracking, large potholes, surface disintegration.
    Rating 1
        Description: The road has completely failed with extensive loss of surface material, large potholes, and patching in failed condition. It is undriveable and needs complete reconstruction.
        Key Characteristics: The road is virtually unusable with deep potholes, surface break-up, and complete structural failure.
        Expected Defects: Large potholes, road disintegration, complete surface failure.
            
    Here are the characteristics of the 10 road surface defects to help visually identify them in images:
        
    1. Ravelling (surface defects)
        Appearance: Surface looks rough with small aggregate particles missing; advanced ravelling shows a worn surface with larger aggregates exposed.
        Key Visual Clue: Loss of surface material, especially in wheel paths or edges.
    2. Bleeding (surface defects)
        Appearance: Shiny, reflective, and often sticky-looking patches on the road, especially in wheel paths.
        Key Visual Clue: Dark, shiny spots that appear wet or oily but are not due to moisture.
    3. Cracking (Longitudinal, Transverse, Reflection, Slippage) (pavement defects)
        Appearance: Longitudinal cracks run parallel to the road center; transverse cracks run perpendicular; slippage cracks are crescent-shaped.
        Key Visual Clue: Linear cracks either along the road or crossing it, with potential for further deterioration.
    4. Rutting (structural distresses)
        Appearance: Deep, long, narrow depressions in the wheel paths.
        Key Visual Clue: Visible grooves or channels where vehicles frequently pass.
    5. Alligator Cracking (structural distresses)
        Appearance: Cracks that form a grid or "chicken wire" pattern resembling alligator skin.
        Key Visual Clue: Small, interconnected cracks, often seen in areas with heavy traffic loading.
    6. Edge Cracking/Breakup (structural distresses)
        Appearance: Cracks that develop along the edges of the pavement, usually within 300mm from the edge.
        Key Visual Clue: Linear cracks or broken pavement material along the road edges.
    7. Surface Distortion (structural distresses)
        Appearance: Wavy or uneven road surface with depressions or bumps, but not necessarily in wheel paths.
        Key Visual Clue: Deformations that create dips, sags, or humps, reducing smoothness.
    8. Patching (structural distresses)
        Appearance: Sections of pavement that have been repaired with new material, often different in color or texture.
        Key Visual Clue: Visible patches with irregular borders; may have cracks or be sinking.
    9. Potholes (structural distresses)
        Appearance: Circular or irregular depressions where part of the pavement has crumbled away.
        Key Visual Clue: Bowl-shaped holes in the surface, sometimes with water or loose debris inside.
    10. Road Disintegration (structural distresses)
        Appearance: Large, rough, and cratered sections of the road where most of the pavement surface is gone.
        Key Visual Clue: Areas where the road surface is severely broken, exposing the underlying layers.

        """

In [5]:
def get_road_condition_grade1(folder_path, model):
    # Iterate through each image in the folder
    grade = []
    img_names = []
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            base64_image = encode_image(image_path)
            # Construct the message with the specified instructions
            messages=[
                    {
                        "role": "system",
                        "content": "You are a pavement engineer tasked with evaluating road conditions from images.\
                        You will also be provided with road condition grading criteria (road_grading_standards) and image.\
                        Use the following step-by-step instructions to evaluate the road grade.\
                        Step 1 – The user will provide you with road condition grading criteria.\
                        Step 2 - The user will provide you with an image in base64 format, focus solely on the asphalt pavement.\
                        Step 3 - Please pay attention to any anomalies on the asphalt pavement, such as surface defects (ravelling,\
                        bleeding), pavement defects(Longitudinal cracks, Transverse crack), structural distresses(alligator crack,\
                        rutting, potholes, surface distortion, edge breakup, patching).\
                        When identifying road anomalies, exclude the effects of shadows and water stains caused by lighting conditions.\
                        Step 4 - Roughly estimate the proportion of each type of anomaly (surface defects, pavement defects and structural\
                        distresses) on the asphalt surface. This ratio will be used as the basis for subsequent road classification.\
                        Step 5 - Using the provided road condition grading criteria determine the grade of the road surface in the image.\
                        Step 6 - Hide your thought process and give me a numerical value. Respond with a single grade number based on the provided standards.\
                        Respond with only one number."},
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": "I will provide you with road condition grading criteria."},
                            {"type": "text", "text": road_grading_standards},
                            {"type": "text", "text": "I will provide you with an image in base64 format, please focus solely on the asphalt pavement"},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                }
                            },
                            {"type": "text", "text": "Please pay attention to any anomalies on the asphalt pavement, such as surface defects (ravelling,\
                            bleeding), pavement defects(Longitudinal cracks, Transverse crack), structural distresses(alligator crack,\rutting, potholes,\
                            surface distortion, edge breakup, patching). \
                            When identifying road anomalies, exclude the effects of shadows and water stains caused by lighting\
                            conditions."},
                            {"type": "text", "text": "Roughly estimate the proportion of each type of anomaly (surface defects, pavement defects and structural distresses)\
                            on the asphalt surface. This ratio will be used as the basis for subsequent road classification."},
                            {"type": "text", "text": "Using the provided road condition grading criteria determine the \
                            grade of the road surface in the image"},
                            {"type": "text", "text": "Hide your thought process and give me a numerical value. Respond with a single grade number based on the provided standards.\
                            Respond with only one number."},
                        ]
                    },
                ]
            
            response = client.chat.completions.create(
                model= model,  # Note: This is a conceptual model name
                messages=messages,
                max_tokens=300,
                temperature=0
            )
            
            try:
                grade_value = int(response.choices[0].message.content)
                grade.append(grade_value)
                img_names.append(image_name)
            except ValueError:
                grade_value = 11
                grade.append(grade_value)
                img_names.append(image_name)
                print(f"Invalid response for {image_name}: {response.choices[0].message.content}")
    return (img_names, grade, )

In [7]:
folder_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\imgs_all"
output = get_road_condition_grade1(folder_path,"gpt-4o")

df = pd.DataFrame({'Name': output[0], 'Model': output[1]})
df['Name'] = df['Name'].str.extract('(\d+)')
csv_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\RC_Model5_o_10131112.csv"#RC_Model3_o6_08252210.csv
df.to_csv(csv_path, index=False)